In [ ]:
import tensorflow as tf
from tensorflow import keras

import os

import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader

import numpy as np
import sklearn
from sklearn.model_selection import train_test_split

import shutil

try:
  shutil.rmtree('logs')
except:
  pass

In [ ]:

frame2 = []
count = 1
for dirname, _, filenames in os.walk('/kaggle/input'):
    for file in filenames:
        if(file.endswith('.csv')):
            temp = pd.read_csv(os.path.join(dirname, file), encoding = "ISO-8859-1")
            temp = torch.Tensor(temp.to_numpy()) 
            #print('file : #', count, '---> ', temp.shape, 'file Name : ', file)
            frame2.append(temp)       
            count += 1

torchFrame2 = torch.cat(frame2)
X = torchFrame2[:,0:23]
Y = torchFrame2[:,23:25]
#X_norm = sklearn.preprocessing.normalize(X, axis=0)
#print(X_norm.shape)
print(Y.shape)

In [ ]:
'''
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X)
X_norm = scaler.transform(X)
print(X_norm.shape)
'''

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_norm = scaler.fit_transform(X)

In [ ]:
for i in range(23):
  print(min(X_norm[:,i]), '    ', max(X_norm[:,i]), '    ',  X_norm[:,i].mean(), '     ', X_norm[:,i].std())

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_norm, Y, test_size=0.2, shuffle=True)
print(X_train.shape)
print(X_test.shape)
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from tensorflow.keras.layers import Input, Dense, LeakyReLU, Dropout

input_ = Input(shape = (23), name='Input')

dense1_shared = Dense(256, name='dense1_shared')(input_)
lr1_shared = LeakyReLU(alpha=0.01, name='lr1_shared')(dense1_shared)
dense2_shared = Dense(512, name='dense2_shared')(lr1_shared)
lr2_shared = LeakyReLU(alpha=0.01, name='lr2_shared')(dense2_shared)
drop1_shared = Dropout(0.05, name='drop1_shared')(lr2_shared)
dense3_shared = Dense(1024, name='dense3_shared')(drop1_shared)
lr3_shared = LeakyReLU(alpha=0.05, name='lr3_shared')(dense3_shared)
drop2_shared = Dropout(0.02, name='drop2_shared')(lr3_shared)
dense4_shared = Dense(256, name='dense4_shared')(drop2_shared)
lr4_shared = LeakyReLU(alpha=0.01, name='lr4_shared')(dense4_shared)

dense1_2 = Dense(256, name='dense1_2')(lr4_shared)
lr1_2 = LeakyReLU(alpha=0.01, name='lr1_2')(dense1_2)
dense2_2 = Dense(512, name='dense2_2')(lr1_2)
lr2_2 = LeakyReLU(alpha=0.01, name='lr2_2')(dense2_2)
drop1_2 = Dropout(0.05, name='drop1_2')(lr2_2)
dense3_2 = Dense(1024, name='dense3_2')(drop1_2)
out2 = Dense(1, activation = 'sigmoid', name = 'out2')(dense3_2)

dense1_3 = Dense(1024, name='dense1_3')(lr4_shared)
lr1_3 = LeakyReLU(alpha=0.01, name='lr1_3')(dense1_3)
drop1_3 = Dropout(0.05, name='drop1_3')(lr1_3)
dense2_3 = Dense(512, name='dense2_3')(drop1_3)
lr2_3 = LeakyReLU(alpha=0.05, name='lr2_3')(dense2_3)
drop2_3 = Dropout(0.02, name='drop2_3')(lr2_3)
dense3_3 = Dense(256, name='dense3_3')(drop2_3)
lr3_3 = LeakyReLU(alpha=0.01, name='lr3_3')(dense3_3)
drop3_3 = Dropout(0.01, name='drop3_3')(lr3_3)
out3 = Dense(7, activation='softmax', name = 'out3')(drop3_3)


model = tf.keras.models.Model(input_, [out2, out3])

In [ ]:
model.compile(
    loss = {
        'out2': 'binary_crossentropy',
        'out3': 'sparse_categorical_crossentropy'
    },
    optimizer='adam',
    metrics = ['accuracy']
)

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model)

In [ ]:
class Logger(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    out2_accuracy = logs.get('out2_accuracy')
    out3_accuracy = logs.get('out3_accuracy')
    val_out2_accuracy = logs.get('val_out2_accuracy')
    val_out3_accuracy = logs.get('val_out3_accuracy')
    print()
    print('='*30, epoch + 1, '='*30)
    print(f'out2_accuracy: {out2_accuracy:.2f}, out3_accuracy: {out3_accuracy:.2f}')
    print(f'val_out2_accuracy: {val_out2_accuracy:.2f}, val_out3_accuracy: {val_out3_accuracy:.2f}')

In [ ]:
_ = model.fit(
      X_train,
      [y_train[:,0], y_train[:,1]],
      batch_size = 32,
      epochs=50, 
      callbacks=[
                 Logger(),
                tf.keras.callbacks.TensorBoard(log_dir='./logs')
      ],
      validation_data=(X_test, [y_test[:,0], y_test[:,1]]),
      verbose=False)

In [ ]:
model.save("my_model.h5")